In [1]:
import torch
from newspaper import Article
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings
warnings.filterwarnings('ignore')
nltk.download('punkt', quiet=True)
import feedparser
from bs4 import BeautifulSoup
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from chatbot import Chat, register_call

In [2]:
from transformers import BertForQuestionAnswering
model_bert = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [3]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [5]:
url = 'https://tfhub.dev/google/universal-sentence-encoder/4'
model_faq = hub.load(url) 

INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 20.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 40.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 60.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 80.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 100.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 120.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 140.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 160.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 180.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 200.00MB
INFO:absl:Downloading https://tfhub.dev/google/univ

In [6]:
dataset = pd.read_excel("C:/Users/susan/MyFolder/BDA/CovidBot/faqs1 (2).xlsx")
state_data = pd.read_csv('C:/Users/susan/MyFolder/BDA/CovidBot/state.csv')
country_data = pd.read_csv('C:/Users/susan/MyFolder/BDA/CovidBot/country.csv')

In [7]:
def answer_question(question, answer_text):
    input_ids = tokenizer.encode(question, answer_text)
    print('Query has {:,} tokens.\n'.format(len(input_ids)))
    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    start_scores, end_scores = model_bert(torch.tensor([input_ids]),token_type_ids=torch.tensor([segment_ids])) 
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer = tokens[answer_start]
    for i in range(answer_start + 1, answer_end + 1):
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        else:
            answer += ' ' + tokens[i]

    return answer

In [8]:
def news_bert(A):
    b = '%20'
    Aa = A+' covid-19'
    for word in Aa:
        if word == ' ':
            Aa = Aa.replace(word,b)
    url = "http://news.google.com/news?q="+Aa+"&hl=en-US&sort=date&gl=IN&num=500&output=rss"
    feed_url = url
    soup = BeautifulSoup()
    text = soup.get_text()
    text = text.replace('\xa0', ' ')
    c=''
    feeds = feedparser.parse(feed_url).entries
    b = feeds[0]['link']
    article = Article(b)
    article.download()
    article.parse()
    article.nlp()
    corpus = article.text
    if len(corpus) > 512:
        corpus = corpus[:512]
    return corpus

In [9]:
def nlp(corpus):
    from textblob.classifiers import NaiveBayesClassifier as NBC
    from textblob import TextBlob
    training_corpus = [
                       ('What is the status of covid in kerela?', 'Class_B'),
                       ("Who is the latest celebrith to be tested positive?", 'Class_B'),
                       ('When will vaccines be found?', 'Class_B'),
                       ('When will colleges open in karnataka', 'Class_B'),
                       ('What is the active case count in Goa?', 'Class_A'),
                       ('Total deaths in bangalore', 'Class_A'),
                       ('number of tested positive cases in the world', 'Class_A')]

    model = NBC(training_corpus) 
    op = model.classify(corpus)
    #print(model.accuracy(test_corpus))
    return op

In [10]:
def embed(input):
    return model_faq([input])
dataset['Question_Vector'] = dataset.Question.map(embed)
dataset['Question_Vector'] = dataset.Question_Vector.map(np.array)
questions = dataset.Question
QUESTION_VECTORS = np.array(dataset.Question_Vector)
COSINE_THRESHOLD = 0.5
def cosine_similarity(v1, v2):
        mag1 = np.linalg.norm(v1)
        mag2 = np.linalg.norm(v2)
        if (not mag1) or (not mag2):
            return 0
        return np.dot(v1, v2) / (mag1 * mag2)
def semantic_search(query, data, vectors):        
        query_vec = np.array(embed(query))
        res = []
        for i, d in enumerate(data):
            qvec = vectors[i].ravel()
            sim = cosine_similarity(query_vec, qvec)
            res.append((sim, d[:100], i))
        return sorted(res, key=lambda x : x[0], reverse=True)      
@register_call("news_faqs")
def news_faqs(query,session_id="general"):
        '''This will return list of all questions according to their similarity,but we'll pick topmost/most relevant question'''
        most_relevant_row = semantic_search(query, questions, QUESTION_VECTORS)[0]
        if most_relevant_row[0][0]>=COSINE_THRESHOLD:
            answer = dataset.Answer[most_relevant_row[2]]
        else:
            answer = answer_question(query, news_bert(query))
        return answer

In [11]:
state_data = pd.read_csv('C:/Users/susan/MyFolder/BDA/CovidBot/state.csv')
country_data = pd.read_csv('C:/Users/susan/MyFolder/BDA/CovidBot/country.csv')
unimportant=['and']
def capitalise(word):
    resp=""
    v = word.split()
    for x in v:
        if x not in unimportant:
            resp += (" " + x.title())
        else:
            resp += (" " + x)

    return resp
import urllib.request, json
with urllib.request.urlopen("https://api.covid19india.org/state_district_wise.json") as url:
    data = json.loads(url.read().decode())
    #print(data)
#distr = ['Andaman and Nicobar Islands','Andhra Pradesh','Arunachal Pradesh','Assam','Bihar','Chandigarh','Chhattisgarh','Delhi','Dadra and Nagar Haveli and Daman and Diu','Goa','Gujarat','Himachal Pradesh','Haryana','Jharkhand','Jammu and Kashmir','Karnataka','Kerala','Ladakh','Lakshadweep','Maharashtra','Meghalaya','Manipur','Madhya Pradesh','Mizoram','Nagaland','Odisha','Punjab','Puducherry','Rajasthan','Sikkim','Telangana','Tamil Nadu','Tripura','Uttar Pradesh','Uttarakhand','West Bengal']
@register_call("country_names")
def country_names(query,session_id="general"):
    query = capitalise(query).strip()
    for i in range(len(country_data)):
        if country_data.iloc[i][1]==query:
            output = 'TotalCases : '+str(country_data.iloc[i][1])+' NewCases : '+str(country_data.iloc[i][2])+' TotalDeaths : '+str(country_data.iloc[i][3])+' NewDeaths : '+str(country_data.iloc[i][4])+' TotalRecovered : '+str(country_data.iloc[i][5])+' ActiveCases : '+str(country_data.iloc[i][6])
            break
        else:
            output = 'Input valid country name'
    return(output)
@register_call("state_names")
def state_names(query,session_id="general"):
    query = capitalise(query).strip()
    for i in range(len(state_data)):
        if state_data.iloc[i][0]==query:
            output = ' Confirmed : '+str(state_data.iloc[i][1])+' Active : '+str(state_data.iloc[i][2])+' Recovered : '+str(state_data.iloc[i][3])+' Deaths : '+str(state_data.iloc[i][4])+' Tested : '+str(state_data.iloc[i][5])
            break
        else:
            output = 'Input valid country name'
    return(output)
@register_call("district_names")
def district_names(query,session_id="general"):
    query = capitalise(query).strip()
    for i in data:
        for j in data[i]['districtData']:
            if j == query:
                output =  ' Active Cases : '+str(data[i]['districtData'][j]['active'])+' Confirmed : '+str(data[i]['districtData'][j]['confirmed'])+' Dead : '+str(data[i]['districtData'][j]['deceased'])+' Recovered : '+str(data[i]['districtData'][j]['recovered'])
    return(output)

In [14]:
firstQuestion="what is death rate in india?"
if nlp(firstQuestion) == 'Class_B':
    temp = "news.template"
else:
    temp = 'demo.template'
Chat(temp).say(firstQuestion)

'The death rate is currently decreasing.'

firstQuestion="count in Haveri"
Chat(temp).say(firstQuestion)

import urllib.request, json
with urllib.request.urlopen("https://api.covid19india.org/state_district_wise.json") as url:
    data = json.loads(url.read().decode())
    #print(data)

distr = ['Andaman and Nicobar Islands','Andhra Pradesh','Arunachal Pradesh','Assam','Bihar','Chandigarh','Chhattisgarh','Delhi','Dadra and Nagar Haveli and Daman and Diu','Goa','Gujarat','Himachal Pradesh','Haryana','Jharkhand','Jammu and Kashmir','Karnataka','Kerala','Ladakh','Lakshadweep','Maharashtra','Meghalaya','Manipur','Madhya Pradesh','Mizoram','Nagaland','Odisha','Punjab','Puducherry','Rajasthan','Sikkim','Telangana','Tamil Nadu','Tripura','Uttar Pradesh','Uttarakhand','West Bengal']

unimportant=['and']
def capitalise(word):
    resp=""
    v = word.split()
    for x in v:
        if x not in unimportant:
            resp += (" " + x.title())
        else:
            resp += (" " + x)
    return resp
def state_names(question):
    question = capitalise(question).strip()
    for i in range(0,len(state_data)):
        for j in question.split(' '):
            if capitalise(j)[1:] == state_data.iloc[i]['Location']:
                output = 'state name : '+ (state_data.iloc[i]['Location'])+'; Active Cases : '+str(state_data.iloc[i][1])+'; New Cases : '+str(state_data.iloc[i][2])+'; Cases per 1 million people : '+str(state_data.iloc[i][4])+'; Deaths : '+str(state_data.iloc[i][5])
                
    else:
        for i in distr:
            #print(data[i]['districtData'])
            for j in data[i]['districtData']:
                for k in question.split(' '):
                    if k == j:
                        output = ' Active Cases : '+str(data[i]['districtData'][j]['active'])+' Confirmed : '+str(data[i]['districtData'][j]['confirmed'])+' Dead : '+str(data[i]['districtData'][j]['deceased'])+' Recovered : '+str(data[i]['districtData'][j]['recovered'])
    return output

def dist_names(question):
    question = capitalise(question).strip()
    for i in distr:
        #print(data[i]['districtData'])
        for j in data[i]['districtData']:
            for k in question.split(' '):
                if k == j:
                    output = ' Active Cases : '+str(data[i]['districtData'][j]['active'])+' Confirmed : '+str(data[i]['districtData'][j]['confirmed'])+' Dead : '+str(data[i]['districtData'][j]['deceased'])+' Recovered : '+str(data[i]['districtData'][j]['recovered'])
    return output

import urllib.request, json
with urllib.request.urlopen("https://api.covid19india.org/data.json") as url:
    data_dist = json.loads(url.read().decode())

for i in data_dist['statewise']:
    print(i)

def state_names(query):
    query = capitalise(query).strip()
    for i in data_dist['statewise']:
        if i['state']==query:
            output = ' Confirmed : '+str(i['confirmed'])+' Active : '+str(i['active'])+' Recovered : '+str(i['recovered'])+' Deaths : '+str(i['deaths'])
            break
        else:
            output = 'Input valid country name'
    return(output)

import urllib.request, json
with urllib.request.urlopen("https://api.covid19api.com/summary") as url:
    data_country = json.loads(url.read().decode())

for i in data_country['Countries']:
    print(i)

def country_names(query):
    query = capitalise(query).strip()
    for i in data_country['Countries']:
        if i['Country']==query:
            output = 'TotalCases : '+str(i['TotalConfirmed'])+' NewCases : '+str(i['NewConfirmed'])+' TotalDeaths : '+str(i['TotalDeaths'])+' NewDeaths : '+str(i['NewDeaths'])+' TotalRecovered : '+str(i['TotalRecovered'])
            break
        else:
            output = 'Input valid country name'
    return(output)

country_names('Armenia')